In [ ]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [ ]:
'''
Loading the dataset
'''

dt = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')

In [ ]:
print(dt.shape)

In [ ]:
corr_matrix = dt.corr()
f, ax = plt.subplots(figsize=(12,10))
sns.heatmap(corr_matrix,vmax=1,vmin=-1,square=True,annot=True)

In [ ]:
sns.pairplot(dt)

In [ ]:
dt.info()
dt.head()

# Feature Engineering
Since we have a lot of categorical features,we need to do feature engineering to make the models understand our dataset.

The first feature we are going to focus is the "affected_roads". This feature lists all the roads that were affected, but if we look closely we can see that most of the affected roads are repetead, and since knowing the name of the road doesn't give us much information, we are going to transform this feature to a numerical by counting the distinct number of roads that were affected.

In [ ]:
test.isna().sum()

In [ ]:
from numpy import NaN


#dt.dropna(inplace=True)
dt.replace(np.NaN,0,inplace=True)

test.replace(np.NaN,0,inplace=True)

In [ ]:
test[test['affected_roads']==0]

In [ ]:
dt.isnull().sum()

In [ ]:
def split_roads(x):
    if not isinstance(x['affected_roads'],int):
        return len(set(filter(None,x['affected_roads'].split(","))))
    else:
        return x['affected_roads']


In [ ]:
dt['affected_roads_num'] = dt.apply(split_roads,axis=1)
test['affected_roads_num'] = test.apply(split_roads,axis=1)

In [ ]:
dt['avg_precipitation'].nunique()

In [ ]:
dt.drop('avg_precipitation', axis=1,inplace=True)
test.drop('avg_precipitation', axis=1,inplace=True)

In [ ]:
dt.drop('affected_roads',axis=1,inplace=True)
test.drop('affected_roads',axis=1,inplace=True)

The next feature we are going to focus is the city name. Let's check what are the possible city names.

In [ ]:
dt['city_name'].nunique()

Since it only has one possible value we can drop this column because it does not add much value to our model.

In [ ]:
dt.drop('city_name',axis=1,inplace=True)
test.drop('city_name',axis=1,inplace=True)

Let's evaluate now the magnitude of delay feature.

In [ ]:
dt['magnitude_of_delay'].value_counts()

We have 3 possible values but the biggest value present is Undefined, this can cause bias to our model.


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
lb = OrdinalEncoder(categories = [['UNDEFINED','MODERATE','MAJOR']])
dt['magnitude_of_delay'] = lb.fit_transform(dt[['magnitude_of_delay']])
test['magnitude_of_delay'] = lb.fit_transform(test[['magnitude_of_delay']])


The next feature we are going to transform is the date feature.Since this is labeled as categorical, we need to transform it to the datetime format.


In [ ]:
dt['record_date'] = pd.to_datetime(dt['record_date'],format='%Y-%m-%d %H:%M')
test['record_date'] = pd.to_datetime(test['record_date'],format='%Y-%m-%d %H:%M')


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

lb_make=OrdinalEncoder(categories = [['DARK','LOW_LIGHT','LIGHT']])
dt['luminosity'] = lb_make.fit_transform(dt[['luminosity']])
test['luminosity'] = lb_make.fit_transform(test[['luminosity']])


In [ ]:
dt['avg_rain'].value_counts()

In [ ]:
avg_rain_lb = OrdinalEncoder(categories=[['Sem Chuva','chuva fraca','chuva moderada','chuva forte']])
dt['avg_rain'] = avg_rain_lb.fit_transform(dt[['avg_rain']])
test['avg_rain'] = avg_rain_lb.fit_transform(test[['avg_rain']])

In [ ]:
target = OrdinalEncoder(categories=[['None','Low','Medium','High','Very_High']])
dt['incidents'] = target.fit_transform(dt[['incidents']])


In [ ]:
dt['week_day'] = dt['record_date'].dt.weekday
dt['month'] = dt['record_date'].dt.month
dt['hour'] = dt['record_date'].dt.hour
dt['day'] = dt['record_date'].dt.day


test['week_day'] = test['record_date'].dt.weekday
test['month'] = test['record_date'].dt.month
test['hour'] = test['record_date'].dt.hour
test['day'] = test['record_date'].dt.day


In [ ]:
dt.drop('record_date',axis=1,inplace=True)
test.drop('record_date',axis=1,inplace=True)

In [ ]:
X_train = dt.drop(['incidents'],axis=1)
y_train = dt['incidents'].to_frame()


## SVC


In [ ]:
from sklearn.svm import SVC
model = SVC(random_state=2022,probability=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=2022)
#model.fit(X_train,y_train)

In [ ]:
predicts = model.predict(test)
test['incidents'] = predicts
test.head()

In [ ]:
params_grid = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel' : ['rbf']}

In [ ]:
from sklearn.model_selection import GridSearchCV, ParameterGrid
svm_model = GridSearchCV(model,params_grid,cv=5,n_jobs=-1,refit=True,verbose=3)


In [ ]:
svm_model.fit(X_train,y_train.values.ravel())

In [ ]:
svm_model.best_score_

## Random Forest

In [ ]:
param_grid = { 
    'n_estimators': [1600,1610],
    'max_features': ['sqrt'],
    'max_depth' : range(15,18),
    'criterion' :['entropy'],
    "min_samples_split": [2,3],
    "min_samples_leaf": [1,2],
    "class_weight": ["balanced_subsample"]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

random_model = RandomForestClassifier(random_state=4000)

randomForest = GridSearchCV(random_model, param_grid = param_grid, n_jobs = -1, cv = 10,verbose=3, scoring='accuracy')
randomForest.fit(X_train,y_train.values.ravel())

In [ ]:
randomForest.best_score_
randomForest.best_params_


In [ ]:
from sklearn.ensemble import RandomForestClassifier
bestModel = RandomForestClassifier(random_state=4000,class_weight='balanced_subsample', criterion='entropy',max_depth=15,min_samples_leaf=1,min_samples_split=2,n_estimators=1610,max_features='sqrt')
bestModel.fit(X_train,y_train.values.ravel())

In [ ]:
predicts = bestModel.predict(test)
test['incidents'] = predicts

In [ ]:
from sklearn.model_selection import GridSearchCV, ParameterGrid


#mean_absolute_error(y_test, predictions)
#mean_squared_error(y_test,predictions)
param_dict={
    "criterion":['gini','entropy'],
    "max_depth":range(1,20),
    "min_samples_split":range(1,20),
    "min_samples_leaf":range(1,20)
}
gs = GridSearchCV(model, param_grid = param_dict,n_jobs=-1,cv=10,scoring='accuracy',verbose=1)
gs.fit(X_train,y_train)


In [ ]:
gs.best_params_
gs.best_estimator_
#gs.best_score_

In [ ]:
best_model=DecisionTreeClassifier(random_state=2022,criterion='entropy',max_depth=13,min_samples_leaf=3,min_samples_split=8)
best_model.fit(X_train,y_train)

In [ ]:
predicts = best_model.predict(test)
test['incidents'] = predicts

In [ ]:
dt.info()

## XGBoost


In [ ]:
param_grid = { 
    'n_estimators': [100,120],
    'learning_rate': [0.2,0.25],
    'gamma': [0],
    'subsample':[0.5,1],
    'min_child_weight': [0,1,2],
    'booster' :['gbtree'],
}

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xg_model = XGBClassifier(random_state=4000)

xg = GridSearchCV(xg_model, param_grid = param_grid, n_jobs = -1, cv = 10,verbose=3, scoring='accuracy')
xg.fit(X_train,y_train.values.ravel())


In [ ]:
xg.best_score_
xg.best_params_


In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
bestModel = XGBClassifier(random_state=4000,booster= 'gbtree',gamma= 0,learning_rate=0.25,min_child_weight= 1,n_estimators=100,subsample=1)
bestModel.fit(X_train,y_train.values.ravel())

In [ ]:
predicts = bestModel.predict(test)
test['incidents'] = predicts

# MLP

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler




In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


In [ ]:
scaler_X = MinMaxScaler(feature_range=(0,1)).fit(X_train)
#scaler_y = MinMaxScaler(feature_range=(0,1)).fit(y_train)
X_scaled = pd.DataFrame(scaler_X.transform(X_train[X_train.columns]),columns=X_train.columns)
#Y_scaled = pd.DataFrame(scaler_y.transform(y_train[y_train.columns]),columns=y_train.columns)
scaler_test = MinMaxScaler(feature_range=(0,1)).fit(test)
test_scaled = pd.DataFrame(scaler_test.transform(test[test.columns]),columns=test.columns)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_train, test_size=0.25, random_state=2022)

In [ ]:
def build_model(activation = 'relu', learning_rate = 0.01):
    model = Sequential()
    model.add(Dense(12, input_dim=12,activation = activation))
    model.add(Dense(20, activation = activation))
    model.add(Dense(5, activation = 'softmax'))
    
    model.compile(
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.optimizers.SGD(learning_rate),
        metrics = ['accuracy','sparse_categorical_accuracy']
    )
    return model

In [ ]:
tuning_dict = {
    'activation' : ['relu'],
    'learning_rate' : [1,0.1,0.01,0.001]
}

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=2022)
model = KerasClassifier(build_fn=build_model, epochs = 200, batch_size=64)
grid_search = GridSearchCV(estimator = model,
                           param_grid= tuning_dict,
                           cv = kf,
                           scoring='accuracy',
                           refit = True,
                           verbose = 1,n_jobs=-1)
grid_search.fit(X_scaled, y_train,validation_split=0.25)

In [ ]:
print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_scaled,y_train,epochs=20)

In [ ]:
from livelossplot import PlotLossesKerasTF

best_model.fit(X_scaled,y_train,validation_split=0.40,epochs=100,callbacks=[PlotLossesKerasTF()],verbose=1)


In [ ]:
predictions = best_model.predict(test_scaled)


In [ ]:
print(predictions)

In [ ]:
y_test_unscaled = pd.DataFrame(scaler_test.inverse_transform(test_scaled))

In [ ]:
y_test_unscaled['incidents'] = predictions

In [ ]:
print(y_test_unscaled)

In [ ]:
test = y_test_unscaled

# Building the output file


In [ ]:
test['incidents'] =  target.inverse_transform(test[['incidents']])

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
test.index +=1
test.reset_index(drop=False,inplace=True)

final_csv = test.to_csv(r'finalt.csv',header=["RowId","Incidents"],columns=['index','incidents'],index=False)

In [ ]:
test.tail()